# Can Dilia be a youth?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import libraries and some constants

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import uproot as ur
import atlas_mpl_style as ampl
ampl.use_atlas_style()

In [3]:
# my worspace and directories

path_prefix = '/home/dportill/LCStudies/'
plotpath = path_prefix+'classifier/Plots/'
modelpath = path_prefix+'classifier/Models/'

In [4]:
# import our resolution utilities

import sys
sys.path.append(path_prefix)
from  util import resolution_util as ru
from  util import plot_util as pu
from  util import ml_util as mu

In [5]:
# Load data v7 with images from pubnote

inputpath = '/fast_scratch/atlas_images/v7/'
rootfiles = ["pi0", "piplus", "piminus"]

#setupPionData(inputpath, rootfiles, branches = []) on ml_util with defaultBranches from ClusterTree
trees, pdata = mu.setupPionData(inputpath, rootfiles)

In [6]:
# pdata -> Dictionary of dataframes. "pi0", "piplus", "piminus" are separated inside of the dictionary

np0 = len(pdata['pi0'])
npp = len(pdata['piplus'])
npm = len(pdata['piminus'])

print("Number of pi0 events: {}".format(np0))
print("Number of pi+ events: {}".format(npp))
print("Number of pi- events: {}".format(npm))
print("Total: {}".format(np0+npp+npm))

Number of pi0 events: 263891
Number of pi+ events: 435967
Number of pi- events: 434627
Total: 1134485


In [7]:
pcells = {
    ifile : {
        layer : mu.setupCells(itree, layer, flatten = False)
        for layer in mu.cell_meta
    }
    for ifile, itree in trees.items()
}

In [8]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow import keras as keras

In [9]:
import tensorflow as tf

In [13]:
##For taking multiple gpus:
#gpu_list = ["/gpu:0","/gpu:1"]
#strategy = tf.distribute.MirroredStrategy(devices=gpu_list)
#ngpu = strategy.num_replicas_in_sync
#print ('Number of devices: {}'.format(ngpu))

In [12]:
# let's just take one gpu:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [22]:
# create train/validation/test subsets containing 70%/10%/20%
# of events from each type of pion event
training_classes = ['pi0','piplus']
pdata_merged, pcells_merged, plabels = mu.createTrainingDatasets(training_classes, pdata, pcells)

 ---------------------------- Index:  0 , label:  pi0
        clusterIndex      truthE  nCluster    clusterE  clusterECalib  \
0                  0  532.180603         1  522.347168     531.995789   
1                  0    7.640409         1    6.733534       8.309240   
2                  0    5.514576         2    2.788937       3.465582   
3                  1    5.514576         2    1.246055       1.994231   
4                  0   24.719658         1   23.665634      25.394232   
...              ...         ...       ...         ...            ...   
263886             0  932.972351         1  908.714111     926.805603   
263887             0    1.626860         1    0.872110       1.478907   
263888             0  475.428680         1  466.900146     480.174408   
263889             0  137.269974         1  132.834503     137.719467   
263890             0    0.329255         1    0.303065       0.652258   

         clusterPt  clusterEta  clusterPhi  cluster_nCells  cluster_s

In [15]:
pcellsE23_EMB2G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (16, 16), layers=['EMB2', 'EMB3']),last=True)

In [16]:
pcellsE1_EMB1G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (128, 4), layers=['EMB1']),last=True)

In [23]:
pcellsT123_T1G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (4, 4), layers=['TileBar0', 'TileBar1', 'TileBar2']),last=True)